# 导入模块

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

In [111]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [112]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [113]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, tprm.contract_price, tprm.purchase_amount
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring   tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- AND tmer.shop_type !=2  -- 剔除商家
and  DATE_FORMAT(om.create_time, '%Y-%m') >= '2022-07'               -- 近15天数据
and DATE_FORMAT(om.create_time, '%Y-%m') <= '2024-09'
-- and  DATE_FORMAT(om.create_time, '%Y-%m') >= DATE_ADD(CURRENT_DATE, INTERVAL -15 DAY)
;
'''

df_order = query(sql1)
df_order.shape

# 提取订单风控数据

# 合并数据

In [ ]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(1558756)

# 处理日期

In [ ]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method',
       'contract_price', 'purchase_amount'],
      dtype='object')

In [ ]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df['下单月份'] = df["create_time"].dt.strftime('%Y-%m')
df['year'] = df["下单日期"].dt.year
df.shape

(1558756, 35)

In [ ]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                                                                                            857185
冰鉴火眸分<500命中强拒                                                                                                                               160181
年龄超过49岁或低于18岁                                                                                                                                87502
命中强拒加入临时黑名单                                                                                                                                  50952
智融分低于458命中强拒                                                                                                                                 21712
                                                                                                                                             ...  
命中云商分低于515强拒(芝麻综合风险等级0豁免), 命中策略240703_4强拒, 命中自有模型强拒, 命中融安分<700                                                   

# 处理备注信息

In [ ]:
df = df[df['sku_attributes'].notnull()]

In [ ]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [ ]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    1518822
1      39922
Name: count, dtype: int64

# 渠道归属

In [ ]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物          794574
搜索渠道          240371
未知渠道          149848
租机宝           108304
小舞直播           90134
               ...  
安逸推                1
数立MAU01            1
数立月活SL11           1
公域推广-活动页推广         1
数立月活SL18           1
Name: count, Length: 141, dtype: int64

# 订单去重

In [ ]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method',
       'contract_price', 'purchase_amount', '下单日期', '月份', '下单月份', 'year',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '免审', '归属渠道'],
      dtype='object')

In [ ]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文", '李珍珍'])].index,inplace=True)

# 定义状态

In [ ]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [ ]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消      697910
前置拦截       271792
机审强拒        99465
客户取消        30191
人审拒绝        25540
出库          24386
已退款         14453
无法联系         3075
出库前风控强拒      3058
待归还          1310
检测中            79
待审核            57
待商家收货           6
Name: count, dtype: int64

# 保留商家的数据

In [ ]:
df_contain = df.copy()

# 剔除商家数据

In [ ]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)


# 去重订单明细导出

In [ ]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-10-31'

In [ ]:

#每日各渠道去重订单统计结果表格导出
# df.to_excel(f"F/myfile/p站数据/订单日报/去重订单明细_{t_date}.xlsx")


# 导入服务订单

In [ ]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(191882, 56)

In [ ]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)



In [ ]:
df["是否出库"].drop_duplicates()
dfsd = df

# 总体转化

In [ ]:
df = df[df['purchase_amount']!=0]

In [ ]:
#总体
df_group = df.groupby(['下单月份']).agg({'order_id':'size','是否进件':'sum', '机审强拒':'sum','人审拒绝':'sum', '是否出库': 'sum'})
df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

df_group["进件出库率"] = df_group["是否出库"]/df_group["是否进件"]
df_group["进件出库率"] = df_group["进件出库率"].apply(lambda x:format(x,".2%"))

df_group = df_group[["去重订单数","是否进件","机审强拒","人审拒绝","风控通过件","风控通过率", '是否出库', '进件出库率']]

df_ck = df[df['是否出库']==1]
df_ck_group = df_ck.groupby('下单月份').agg({'new_actual_money': 'sum'})
# df_ck_group = df_ck.groupby('下单月份').agg({'new_actual_money': 'sum'}) 'contract_price': 'sum',  
df_groups = pd.merge(df_group, df_ck_group, on='下单月份', how='inner')
df_groups[['去重订单数', '是否进件', '是否出库', '进件出库率', 'new_actual_money']]
# df_groups

,去重订单数,是否进件,是否出库,进件出库率,new_actual_money
下单月份,,,,,
2022-07,2048,283,41,14.49%,186985.31
2022-08,4897,1371,108,7.88%,531245.23
2022-09,5956,1534,111,7.24%,769982.82
2022-10,4503,1140,170,14.91%,840414.50
2022-11,5873,725,61,8.41%,461417.06
2022-12,5424,1097,75,6.84%,758928.46
2023-01,20045,2771,232,8.37%,2223392.80
2023-02,28696,3875,335,8.65%,3331643.98
2023-03,30763,3199,316,9.88%,3174162.57


# 平均值

In [ ]:
df.groupby('year').agg({'是否出库': 'sum', 'purchase_amount': 'mean', 'new_actual_money': 'mean'})

,是否出库,purchase_amount,new_actual_money
year,,,
2022,566,7118.212000,9737.490001
2023,9014,7303.059260,10561.755656
2024,14647,7337.060387,11261.237124
